In [1]:
import re
import nltk
import numpy as np
import pandas as pd
import requests
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\Ambarish
[nltk_data]     Deb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Ambarish Deb\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to C:\Users\Ambarish
[nltk_data]     Deb\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to C:\Users\Ambarish
[nltk_data]     Deb\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Ambarish
[nltk_data]     Deb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Dropping duplicates

In [2]:
resumedata = pd.read_csv("C://Users//Ambarish Deb//Downloads//cleaned resume data.csv")
resumedata=resumedata.drop_duplicates(['cleaned data'])
resumedata

,Unnamed: 0,category,cleaned data
0,0,Data Science and engineering,expertise data and quantitative analysis decis...
4,4,HR,technical skills typewriting tora spsseducatio...
5,5,HR,education details ba mumbai university hr skil...
6,6,HR,education details mba acn college of engineeri...
7,7,HR,key skills computerized a ounting with tally s...
...,...,...,...
1594,2178,Digital Media Management,digital advertising consultant experience dig...
1595,2179,Digital Media Management,digital communications specialist summary dig...
1596,2180,Digital Media Management,digital marketing a ount manager experience t...
1597,2181,Digital Media Management,digital marketing manager career focus digita...


### Function to extract names

In [3]:
def extract_names(txt):
    person_names = []
    for sent in nltk.sent_tokenize(txt):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label') and chunk.label() == 'PERSON':
                person_names.append(
                    ' '.join(chunk_leave[0] for chunk_leave in chunk.leaves())
                )
    return person_names

### Function to extract phone number

In [4]:
PHONE_REG = re.compile(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]')
def extract_phone_number(txt):
    phone = re.findall(PHONE_REG, txt)
    if phone:
        number = ''.join(phone[0])
        if txt.find(number) != 0 and len(number) == 10:
            return number
    return None

### Function to extract email

In [5]:
EMAIL_REG = re.compile(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+')
def extract_emails(txt):
    return re.findall(EMAIL_REG, txt)

### Function to extract skills
The SKILLS list contains all the skills that are being searched for and are relevant to the company. Here, the skills taken are all from data science and engineering domains.

The extract skills function parses the text and sees if there are any matches in the resume.If there are, it appends that skill to the skills column in that row.

In [6]:
SKILLS = [
    'Data Science','matlab','tableau','retail analytics','informatica','financial analytics','powerbi','nlp','Machine Learning','decision analytics', 'predictive modeling',  "kpi", "dashboards","power query", "big data","queries","data mining", "visualization", "business intelligence", 'forecasts',"Deep Learning",
    'Python','c','c++','java','javascript','pandas'
    'Excel',
    'Project Management',
    'Consulting',
    'DevOps', 'big data',
    'Hadoop','hdfs','hive','sqoop','apache','spark','scala','shell scripting','linux','spark','pyspark','cloud'
    'Database',
    'Git','GitHub','Version control','source control'
    'SQL','oracle','mysql','javascript','html','ajax','jquery','xml','agile methodology','devops methodology','scrum','jira','git','bitbucket','angular js ',
    'NOSQL',
    'Quantitative Analysis',
    'Decision Making'  
]
def extract_skills(input_text):
    stop_words = set(nltk.corpus.stopwords.words('english'))
    word_tokens = nltk.tokenize.word_tokenize(input_text)
    filtered_tokens = [w for w in word_tokens if w not in stop_words]
    filtered_tokens = [w for w in word_tokens if w.isalpha()]
    bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtered_tokens, 2, 3)))
    found_skills = set()
    for token in filtered_tokens:
        if token.lower() in map(lambda x:x.lower(),SKILLS):
            found_skills.add(token)
    for ngram in bigrams_trigrams:
        if ngram.lower() in SKILLS:
            found_skills.add(ngram)
    return found_skills

### Extracting degrees
Similar method is followed to extract degrees. All relevant degrees are mentioned in the list and the parser checks for matches in the text.

In [7]:
degrees = [
    'btech','b tech','bsc','b.sc','b sc', 'mca', 'mba','msc','m sc', 'phd', 'bachelor of engineering','bachelor of science'  
]

def extract_degrees(input_text):
    stop_words = set(nltk.corpus.stopwords.words('english'))
    word_tokens = nltk.tokenize.word_tokenize(input_text)
    filtered_tokens = [w for w in word_tokens if w not in stop_words]
    filtered_tokens = [w for w in word_tokens if w.isalpha()]
    bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtered_tokens, 2, 3)))
    found_deg = set()
    for token in filtered_tokens:
        if token.lower() in map(lambda x:x.lower(),degrees):
            found_deg.add(token)
    for ngram in bigrams_trigrams:
        if ngram.lower() in degrees:
            found_deg.add(ngram)
    return found_deg

In [8]:
resumedata.head()

,Unnamed: 0,category,cleaned data
0,0,Data Science and engineering,expertise data and quantitative analysis decis...
4,4,HR,technical skills typewriting tora spsseducatio...
5,5,HR,education details ba mumbai university hr skil...
6,6,HR,education details mba acn college of engineeri...
7,7,HR,key skills computerized a ounting with tally s...


In [9]:
resumedata['category'].value_counts()

Business Development            240
HR                              119
Consultant                      115
Graphic Design                  106
Digital Media Management         95
Data Science and engineering     22
Customer Support                 21
Software Development             19
QA                               14
Database                         12
Management                        7
DevOps                            7
Business Analytics                6
ERP                               6
Cybersecurity                     5
Web Development                   4
Name: category, dtype: int64

In [10]:
del resumedata["Unnamed: 0"]
resumedata['index']=[i for i in range(len(resumedata['category']))]

In [11]:
resumedata

,category,cleaned data,index
0,Data Science and engineering,expertise data and quantitative analysis decis...,0
4,HR,technical skills typewriting tora spsseducatio...,1
5,HR,education details ba mumbai university hr skil...,2
6,HR,education details mba acn college of engineeri...,3
7,HR,key skills computerized a ounting with tally s...,4
...,...,...,...
1594,Digital Media Management,digital advertising consultant experience dig...,793
1595,Digital Media Management,digital communications specialist summary dig...,794
1596,Digital Media Management,digital marketing a ount manager experience t...,795
1597,Digital Media Management,digital marketing manager career focus digita...,796


### Selecting resumes related to Data Science, Engineering & Databases 

In [12]:
DS_DB_RESUMES=resumedata.loc[resumedata['category'].isin(["Data Science and engineering","Database"])]
DS_DB_RESUMES=DS_DB_RESUMES.reset_index()
del DS_DB_RESUMES['level_0']
del DS_DB_RESUMES['index']

In [13]:
DS_DB_RESUMES['skills']=DS_DB_RESUMES['cleaned data'].apply(lambda x: extract_skills(x))
DS_DB_RESUMES['skills']=DS_DB_RESUMES['skills'].apply(lambda x: ' , '.join(x))

In [14]:
DS_DB_RESUMES['qualifications']=DS_DB_RESUMES['cleaned data'].apply(lambda x: extract_degrees(x))
DS_DB_RESUMES['qualifications']=DS_DB_RESUMES['qualifications'].apply(lambda x: ' , '.join(x))

In [15]:
DS_DB_RESUMES

,category,cleaned data,skills,qualifications
0,Data Science and engineering,expertise data and quantitative analysis decis...,"tableau , predictive modeling , big data , for...",
1,Database,technical expertise db languages sql database ...,"linux , oracle",
2,Database,education details january bsc mumbai maharasht...,"c , linux , queries , mysql",bsc
3,Database,education details may to may bachelor of scien...,"c , linux , oracle",bachelor of science
4,Data Science and engineering,education details january bachelor of engineer...,"jira , informatica , oracle","bsc , bachelor of engineering"
5,Data Science and engineering,skill set talend big data informatica power ce...,"big data , xml , informatica , mysql , queries",
6,Data Science and engineering,computer skills yes sql knowledge yes unix kno...,"c , informatica",
7,Data Science and engineering,skills programming languages python pandas num...,"tableau , mysql , jquery , javascript , git , ...",
8,Data Science and engineering,education details may to may b e uit rgpv data...,"github , python",
9,Data Science and engineering,areas of interest deep learning control system...,"github , mysql , matlab , linux , java , python",b tech


In [16]:
DS_DB_RESUMES['mail']=DS_DB_RESUMES['cleaned data'].apply(lambda x: extract_emails(x))
DS_DB_RESUMES['mail']=DS_DB_RESUMES['mail'].apply(lambda x: ' , '.join(x))

### Final Parsed Content
It is observed that the last 2 columns are empty. It's because names and mail ids weren't mentioned in the resumes.
    1. If we were to widen the scope of this analysis to other job roles, it would be best to keep these columns as other resumes may mention those details.
    2. Else, we can delete these columns as they provide no insight.

In [17]:
DS_DB_RESUMES

,category,cleaned data,skills,qualifications,mail
0,Data Science and engineering,expertise data and quantitative analysis decis...,"tableau , predictive modeling , big data , for...",,
1,Database,technical expertise db languages sql database ...,"linux , oracle",,
2,Database,education details january bsc mumbai maharasht...,"c , linux , queries , mysql",bsc,
3,Database,education details may to may bachelor of scien...,"c , linux , oracle",bachelor of science,
4,Data Science and engineering,education details january bachelor of engineer...,"jira , informatica , oracle","bsc , bachelor of engineering",
5,Data Science and engineering,skill set talend big data informatica power ce...,"big data , xml , informatica , mysql , queries",,
6,Data Science and engineering,computer skills yes sql knowledge yes unix kno...,"c , informatica",,
7,Data Science and engineering,skills programming languages python pandas num...,"tableau , mysql , jquery , javascript , git , ...",,
8,Data Science and engineering,education details may to may b e uit rgpv data...,"github , python",,
9,Data Science and engineering,areas of interest deep learning control system...,"github , mysql , matlab , linux , java , python",b tech,
